<a href="https://colab.research.google.com/github/mololuwaf/home-sales/blob/main/HomeSales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.3.1'
spark_version = 'spark-3.4.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()


Hit:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:4 http://archive.ubuntu.com/ubuntu focal-updates InRelease
Hit:5 http://archive.ubuntu.com/ubuntu focal-backports InRelease
Hit:6 http://security.ubuntu.com/ubuntu focal-security InRelease
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:9 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Reading package lists... Done


In [8]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [9]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
sales_df = spark.read.csv(SparkFiles.get('home_sales_revised.csv'), sep=',',header=True)
sales_df.show(5)

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
+--------------------+----------+----------+------+--------+---------+----------

In [10]:
# 2. Create a temporary view of the DataFrame.
sales_df.createOrReplaceTempView('sales')

In [11]:
spark.sql("""SELECT date_built, round(avg(price), 2) AS Avg_Price_4_Bed
          FROM sales 
          WHERE bedrooms = 4
          GROUP BY date_built
          ORDER BY date_built""").show()

+----------+---------------+
|date_built|Avg_Price_4_Bed|
+----------+---------------+
|      2010|      296800.75|
|      2011|       302141.9|
|      2012|      298233.42|
|      2013|      299999.39|
|      2014|      299073.89|
|      2015|      307908.86|
|      2016|      296050.24|
|      2017|      296576.69|
+----------+---------------+



In [12]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
spark.sql("""SELECT date_built, round(avg(price), 2) AS Avg_Price_3Bed_3Bath
          FROM sales 
          WHERE bedrooms = 3 AND bathrooms=3
          GROUP BY date_built
          ORDER BY date_built""").show()

+----------+--------------------+
|date_built|Avg_Price_3Bed_3Bath|
+----------+--------------------+
|      2010|           292859.62|
|      2011|           291117.47|
|      2012|           293683.19|
|      2013|           295962.27|
|      2014|           290852.27|
|      2015|            288770.3|
|      2016|           290555.07|
|      2017|           292676.79|
+----------+--------------------+



In [13]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
spark.sql("""SELECT date_built, round(avg(price), 2) AS 3Bed_3Bath_2Floor
          FROM sales 
          WHERE bedrooms = 3
          AND bathrooms = 3
          AND floors = 2
          AND sqft_living > 2000
          GROUP BY date_built
          ORDER BY date_built""").show()

+----------+-----------------+
|date_built|3Bed_3Bath_2Floor|
+----------+-----------------+
|      2010|        285010.22|
|      2011|        276553.81|
|      2012|        307539.97|
|      2013|        303676.79|
|      2014|        297619.46|
|      2015|        297609.97|
|      2016|         293965.1|
|      2017|        280527.62|
+----------+-----------------+



In [17]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()
spark.sql("""SELECT round(avg(view), 2) 
          FROM sales 
          WHERE price >= 350000
          """).show()



print("--- %s seconds ---" % (time.time() - start_time))

+-------------------+
|round(avg(view), 2)|
+-------------------+
|              32.26|
+-------------------+

--- 0.5035855770111084 seconds ---


In [18]:
# 7. Cache the the temporary table home_sales.
spark.sql('cache table sales')

DataFrame[]

In [20]:
# 8. Check if the table is cached.
spark.catalog.isCached('sales')


True

In [22]:
# 9. Using the cached data, run the query that filters out the view ratings with average price 
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()
spark.sql("""SELECT round(avg(view), 2) 
          FROM sales 
          WHERE price >= 350000
          """).show()


print("--- %s seconds ---" % (time.time() - start_time))

+-------------------+
|round(avg(view), 2)|
+-------------------+
|              32.26|
+-------------------+

--- 0.7865958213806152 seconds ---


In [23]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data 
sales_df.write.partitionBy('date_built').mode('overwrite').parquet('sales_partitioned')

In [25]:
# 11. Read the formatted parquet data.
partition_df = spark.read.parquet('sales_partitioned')

In [26]:
# 12. Create a temporary table for the parquet data.
partition_df.createOrReplaceTempView('sales_partitioned')

In [30]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000 
# with the parquet DataFrame. Round your average to two decimal places. 
# Determine the runtime and compare it to the cached version.

start_time = time.time()
spark.sql("""SELECT round(avg(view), 2) 
          FROM sales_partitioned 
          WHERE price >= 350000
          """).show()


print("--- %s seconds ---" % (time.time() - start_time))


+-------------------+
|round(avg(view), 2)|
+-------------------+
|              32.26|
+-------------------+

--- 0.7972714900970459 seconds ---


In [31]:
# 14. Uncache the home_sales temporary table.
spark.sql('uncache table sales')

DataFrame[]

In [32]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached('sales')

False